In [1]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE

# 경고 메시지 무시
warnings.filterwarnings('ignore')

print("필요한 라이브러리 임포트 완료")

필요한 라이브러리 임포트 완료


In [2]:
# 1. 데이터 로드
print("1. 'combined_output.csv' 파일 로딩 중...")
try:
    # notebooks 폴더에 있으므로, 상위 폴더의 csv를 로드합니다.
    df = pd.read_csv('../combined_output.csv', low_memory=False)
    print(f"데이터 로드 완료: {df.shape[0]} 행, {df.shape[1]} 열")
    print("\n데이터 샘플 (상위 5개 행):")
    print(df.head())
except FileNotFoundError:
    print("오류: '../combined_output.csv' 파일을 찾을 수 없습니다.")
    print("스크립트가 'notebooks' 폴더 내에 있고, CSV 파일이 프로젝트 루트에 있는지 확인하세요.")
    df = None # 오류 발생 시 df를 None으로 설정

1. 'combined_output.csv' 파일 로딩 중...
데이터 로드 완료: 51872 행, 1200 열

데이터 샘플 (상위 5개 행):
            mod_d             ID       ID_fam    year  region  town_t  apt_t  \
0  b'2023.01.13.'  b'A601169401'  b'A6011694'  2017.0     1.0     1.0    1.0   
1  b'2023.01.13.'  b'A601169402'  b'A6011694'  2017.0     1.0     1.0    1.0   
2  b'2023.01.13.'  b'A601177901'  b'A6011779'  2017.0     1.0     1.0    1.0   
3  b'2023.01.13.'  b'A601177902'  b'A6011779'  2017.0     1.0     1.0    1.0   
4  b'2023.01.13.'  b'A601177903'  b'A6011779'  2017.0     1.0     1.0    1.0   

       psu  sex   age  ...  HE_CNfh22_1  HE_CNfh22_2  HE_CNfh22_3  \
0  b'A601'  1.0  76.0  ...          NaN          NaN          NaN   
1  b'A601'  2.0  69.0  ...          NaN          NaN          NaN   
2  b'A601'  1.0  39.0  ...          NaN          NaN          NaN   
3  b'A601'  2.0  35.0  ...          NaN          NaN          NaN   
4  b'A601'  1.0   1.0  ...          NaN          NaN          NaN   

   HE_CNfh22_4  HE_CNf

In [3]:
# 2. 특징 및 목표 변수 선택
if df is not None:
    print("\n2. 모델링에 사용할 변수 선택 중...")
    # 논문 목표에 맞춰 비침습적 특징 변수 목록 사용
    features = [
        'age', 'sex', 'D_1_1', 'BOI', 'BOI_2', 
        'HE_ht', 'HE_wt', 'HE_wc', 'HE_BMI', 'HE_sbp', 'HE_dbp', 
        'BP16_1', 'BP16_2', 'BS3_2', 'BS12_47', 'BS12_2', 'HE_fh'
    ]
    target = 'BS3_1' # 흡연 여부
    
    # 필요한 모든 컬럼이 데이터프레임에 있는지 확인
    required_columns = features + [target]
    missing_cols = [col for col in required_columns if col not in df.columns]
    if missing_cols:
        print(f"오류: CSV 파일에 다음 필수 컬럼이 없습니다: {missing_cols}")
        df_analysis = None
    else:
        df_analysis = df[required_columns].copy()
        print(f"분석에 사용할 {len(required_columns)}개 컬럼 선택 완료.")
else:
    df_analysis = None


2. 모델링에 사용할 변수 선택 중...
오류: CSV 파일에 다음 필수 컬럼이 없습니다: ['BOI', 'BOI_2']


In [4]:
# 3. 목표 변수 생성 (흡연 여부)
if df_analysis is not None:
    print("\n3. 목표 변수 'smoking' 생성 중...")
    # BS3_1: 1(매일), 2(가끔) -> 1(흡연자) / 그 외 -> 0(비흡연자)
    df_analysis['smoking'] = df_analysis[target].apply(lambda x: 1 if x in [1, 2] else 0)
    df_analysis = df_analysis.drop(target, axis=1)
    
    print('목표 변수 분포:')
    print(df_analysis['smoking'].value_counts(normalize=True))

In [5]:
# 4. 결측치 처리 (중앙값 대체)
if df_analysis is not None:
    print("\n4. 결측치 처리 중...")
    for col in df_analysis.columns:
        if df_analysis[col].isnull().any():
            median_val = df_analysis[col].median()
            df_analysis[col].fillna(median_val, inplace=True)
    print('결측치 처리 완료. 결측치 확인:')
    print(df_analysis.isnull().sum())

In [6]:
# 5. 모델 학습 데이터 분할
if df_analysis is not None:
    print("\n5. 훈련/테스트 데이터 분할 중...")
    X = df_analysis.drop('smoking', axis=1)
    y = df_analysis['smoking']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    print(f"훈련 데이터 크기: {X_train.shape}, 테스트 데이터 크기: {X_test.shape}")

In [7]:
# 6. SMOTE 오버샘플링 적용
if 'X_train' in locals() and 'y_train' in locals():
    print("\n6. SMOTE 오버샘플링 적용 중...")
    smote = SMOTE(random_state=42)
    X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
    print("SMOTE 적용 완료. 훈련 데이터 분포:")
    print(y_train_res.value_counts())

In [8]:
# 7. 랜덤 포레스트 모델 훈련
if 'X_train_res' in locals() and 'y_train_res' in locals():
    print("\n7. 랜덤 포레스트 모델 훈련 중...")
    model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    model.fit(X_train_res, y_train_res)
    print("모델 훈련 완료.")

In [9]:
# 8. 모델 평가
if 'model' in locals() and 'X_test' in locals() and 'y_test' in locals():
    print("\n8. 모델 성능 평가 중...")
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"모델 정확도: {accuracy:.4f}")
    print("\n[성능 평가 리포트]:")
    print(classification_report(y_test, y_pred, target_names=['Non-smoker (0)', 'Smoker (1)']))
    print("\n[혼동 행렬 (Confusion Matrix)]:")
    print(confusion_matrix(y_test, y_pred))

In [11]:
# 9. 변수 중요도 시각화
if 'model' in locals() and 'X' in locals():
    print("\n9. 변수 중요도 시각화 중...")
    feature_importances = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
    
    plt.figure(figsize=(12, 8))
    sns.barplot(x=feature_importances, y=feature_importances.index)
    plt.title('흡연 예측 모델의 변수 중요도 (SMOTE 적용)')
    plt.plt.xlabel('중요도')
    plt.ylabel('변수')
    plt.tight_layout()
    plt.show()
    print("변수 중요도 그래프 출력 완료.")